In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from collections import Counter
from pyvis.network import Network

C:\ProgramData\Anaconda3\envs\IS_596\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def data_processing(sub, field = "submission", info=False):
    file_name = "data/" + field + "_" + sub + ".csv"
    df = pd.read_csv(file_name,index_col=0)
    if info:
        print("subreddit : " + sub)
        print(df.describe())
        print(df.info())
    return df

In [5]:
def have_bidirectional_relationship(G, node1, node2):
    
    return G.has_edge(node1, node2) and G.has_edge(node2, node1)

In [6]:
def author_extraction(df_submission, df_comment):
    df_comment.dropna(subset=['author'], inplace = True)
    author = dict(zip(df_comment.id, df_comment.author))
    df_submission.dropna(subset=['author'], inplace = True)
    df_submission.reset_index(inplace=True)
    author.update(zip(df_submission.id, df_submission.author))
    return author

In [7]:
def relation_extraction(df, author):
    relation = []
    
    top_level_df = df[df['parent_id'].str.contains("t3_")].dropna()
    not_top_level_df = df[df['parent_id'].str.contains("t1_")].dropna()
    
    for index, row in top_level_df.iterrows():
        try:
            parent_author = author[row["parent_id"][3:]]
            relation.append((row["author"], parent_author))
        except:
            pass
    
    for index, row in not_top_level_df.iterrows():
        try:
            parent_author = author[row["parent_id"][3:]]
            relation.append((row["author"], parent_author))
        except:
            pass
    
    return relation

In [8]:
def network_extraction(sub):
    comment_df = data_processing(sub, field = "comment")
    submission_df = data_processing(sub)
    author_dict = author_extraction(submission_df, comment_df)
    relation = relation_extraction(comment_df, author_dict)
    return relation

In [9]:
def network_to_df(net, threshold = 6):
    counter = Counter(net)
    counter = Counter({k: c for k, c in counter.items() if c >= threshold})
    common_relation = sorted(counter, key=counter.get, reverse=True)
    
    relation_df = pd.DataFrame.from_dict(counter, orient='index').reset_index()
    relation_df[['target','source']] = pd.DataFrame(relation_df['index'].tolist(),index=relation_df.index)
    relation_df = relation_df[relation_df['source'] != relation_df['target']]
    relation_df.drop(['index'], axis = 1, inplace = True)
    relation_df.rename(columns = {0:'weight'}, inplace = True)
    relation_df = relation_df[['source', 'target', 'weight']]
    
    return relation_df

In [10]:
def network_builder(relation_df, threshold = 5, net_type = "simple", sub = None):
    filename = 'output/r_' + sub + '_' + net_type + '.html'
    
    G = nx.from_pandas_edgelist(relation_df, 'source', 'target', 'weight')
    
    for component in list(nx.connected_components(G)):
        if len(component) < threshold:
            for node in component:
                G.remove_node(node)
    
    net = Network(height="750px", width="100%", bgcolor="#222222", 
                     font_color="white", notebook = True, cdn_resources = 'remote')

    # set the physics layout of the network
    net.barnes_hut()
    net.from_nx(G)
    try:
        net.show(filename)
    except:
        pass

In [11]:
def mutual_network_builder(relation_df, threshold = 10, sub = None):
    filename = 'output/r_' + sub + '_mutual.html'
    
    G = nx.from_pandas_edgelist(relation_df, 'source', 'target', 'weight', 
                                create_using = nx.DiGraph())
        
    biconnections = []

    for u, v, w in G.edges.data("weight"):
        if u > v:
            v, u = u, v
        if have_bidirectional_relationship(G, u, v):
            if u != v:
                biconnections.append((u, v, w))
        
    G_02 = nx.DiGraph()
    G_02.add_nodes_from(relation_df["source"])
    G_02.add_weighted_edges_from(biconnections)
    G_02.remove_nodes_from(list(nx.isolates(G_02)))
    
    for component in list(nx.weakly_connected_components(G_02)):
        if len(component) < threshold:
            for node in component:
                G_02.remove_node(node)
                
    remains = []

    for u, v, w in G_02.edges.data("weight"):
        remains.append((u, v, w))
        
        
    mut_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")

    # set the physics layout of the network
    mut_net.barnes_hut()

    for e in remains:
        src = e[0]
        dst = e[1]
        w = e[2]

        mut_net.add_node(src, src, title=src, color = "orange")
        mut_net.add_node(dst, dst, title=dst, color = "orange")
        mut_net.add_edge(src, dst, value=w, color = "orange")

    neighbor_map = mut_net.get_adj_list()

    # add neighbor data to node hover data
    for node in mut_net.nodes:
                    node["title"] += " Neighbors:<br>" + "<br>".join(neighbor_map[node["id"]])
                    node["value"] = len(neighbor_map[node["id"]])

    try:
        mut_net.show(filename)
    except:
        pass


In [12]:
relation = network_extraction("politics")

In [13]:
relation_df = network_to_df(relation, threshold = 6)
network_builder(relation_df, sub = "politics")

In [14]:
relation_df = network_to_df(relation, threshold = 2)
mutual_network_builder(relation_df, sub = "politics", threshold = 20)